In [1]:
import sys
import util
from Polygenic_score import *

import os
import subprocess
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics
import re

/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
def write_cov_file(pca_file,nb_pc):

    cmd_sed = "sed -i -e \"s/:/ /g\" -e \"s/\s\+/\t/g\" -e \"s/^\s\+//g\" {}".format(pca_file)
    p = subprocess.Popen(cmd_sed, shell=True)
    assert(p.wait()==0)
    
    cov_file = pca_file[:-9]+".cov"
    a=""
    for i in range(nb_pc):
        a=a+("\tPC{}".format(i+1))
    

    cmd_print = "printf \"FID\tIID{}\n\" > {}".format(a,cov_file)

    p = subprocess.Popen(cmd_print, shell=True)
    assert(p.wait()==0)

    cmd_tail = "tail -n+2 {} | cut -f1-{} >> {}".format(pca_file, 2+nb_pc, cov_file) 

    #-n+2 récup lignes en commançant par le 2ème
    #-f1-2+nb_pc résup les champs 1-nb_pc+2
    
    p = subprocess.Popen(cmd_tail, shell=True)
    assert(p.wait()==0)
    
    print "Wrote {}".format(cov_file)

In [21]:
def save_strat(strat_file, strat):
    pca_res = pd.read_table(strat_file, delim_whitespace=True, skiprows=1, header=None)
    PCs_out = strat + ".PCs"
    with open(PCs_out, "a") as f:
        f.write(', '.join(str(pca_res[0].values)))
        f.write('\n')
        f.write(str(pca_res[2].values.tolist())[1:-1])
        f.write('\n')
        f.write(str(pca_res[3].values.tolist())[1:-1])
        f.write('\n')


In [38]:
def plink_prune(plink_bfile,outfile):

    cmd_prune = """plink --bfile {} --exclude /home/vcabeli/Documents/data/high-LD-regions_37.txt --range --indep-pairwise 50 5 0.2 --allow-extra-chr --out {}""".format(plink_bfile, outfile+"_prune")
    p = subprocess.Popen(cmd_prune, shell=True)
    assert(p.wait() == 0)

    cmd_extract = """plink --bfile {} --extract {} --allow-extra-chr --chr 1-23 --make-bed --out {}""".format(plink_bfile, outfile+"_prune.prune.in", 
                              outfile+"_pruned")
    p = subprocess.Popen(cmd_extract, shell=True)
    assert(p.wait() == 0)

    print "Wrote {} bed/bim/fam.".format( outfile+"_pruned")


In [5]:
snp_data, pheno = util.load_data("/home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR")

Using last column in .fam file as phenotype.
	Considering phenotype as a plink binary 1/2.
	Found 84 cases and 693 controls.
Loaded 777 samples and 272115 SNPs.


In [43]:
plink_prune("/home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR", "GRIV_DESIR_PR_low_ld")
cmd_smart_pca2 = "smartpca.perl -i {} -a {} -b {} -o {} -p {} -e {} -l {} -k {} -t {} -m {}".format("GRIV_DESIR_PR_low_ld_pruned.bed",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.bim",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.fam",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.pca",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.plot",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.eval",
                                                                                               "GRIV_DESIR_PR_low_ld_pruned.log",
                                                                                               2, 2, 0)
print cmd_smart_pca2
p = subprocess.Popen(cmd_smart_pca2, shell=True)
p.wait()


write_cov_file("GRIV_DESIR_PR_low_ld_pruned.pca.evec",2)
save_strat("GRIV_DESIR_PR_low_ld_pruned.pca.evec", "GRIV_DESIR_PR_low_ld{}".format(5)) #OUTPUT LINE


Wrote GRIV_DESIR_PR_low_ld_pruned bed/bim/fam.
smartpca.perl -i GRIV_DESIR_PR_low_ld_pruned.bed -a GRIV_DESIR_PR_low_ld_pruned.bim -b GRIV_DESIR_PR_low_ld_pruned.fam -o GRIV_DESIR_PR_low_ld_pruned.pca -p GRIV_DESIR_PR_low_ld_pruned.plot -e GRIV_DESIR_PR_low_ld_pruned.eval -l GRIV_DESIR_PR_low_ld_pruned.log -k 2 -t 2 -m 0
Wrote GRIV_DESIR_PR_low_ld_pruned.cov


In [46]:
#test d'association: classification (Rlog) NO COVAR
cmd_second_gwas = "plink --bfile {} --logistic --out {}".format("/home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR",
                                                                                                   "GRIV_DESIR_PR_nocovar")
print cmd_second_gwas
p = subprocess.Popen(cmd_second_gwas, shell=True)
p.wait()
low_ld_res = pd.read_table("GRIV_DESIR_PR_nocovar.assoc.logistic", delim_whitespace=True)
low_ld_res.head()


plink --bfile /home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR --logistic --out GRIV_DESIR_PR_nocovar


,CHR,SNP,BP,A1,TEST,NMISS,OR,STAT,P
0,1,rs3737728,1021415,A,ADD,774,1.0490,0.2531,0.8002
1,1,rs6687776,1030565,T,ADD,777,1.2220,0.9701,0.3320
2,1,rs9651273,1031540,A,ADD,776,0.9359,-0.3393,0.7344
3,1,rs4970405,1048955,G,ADD,774,0.9345,-0.2642,0.7916
4,1,rs12726255,1049950,G,ADD,777,1.0810,0.3591,0.7195


In [47]:
cmd_second_gwas = "plink --bfile {} --covar {}  --hide-covar --logistic --out {}".format("/home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR",
                                                                                                   "GRIV_DESIR_PR_low_ld_pruned.cov",
                                                                                                   "low_ld")
print cmd_second_gwas
p = subprocess.Popen(cmd_second_gwas, shell=True)
p.wait()
low_ld_res = pd.read_table("low_ld.assoc.logistic", delim_whitespace=True)
low_ld_res.head()

plink --bfile /home/vcabeli/Documents/analyses_current/GRIV_DESIR/GRIV_DESIR_PR --covar GRIV_DESIR_PR_low_ld_pruned.cov  --hide-covar --logistic --out low_ld


,CHR,SNP,BP,A1,TEST,NMISS,OR,STAT,P
0,1,rs3737728,1021415,A,ADD,774,0.9147,-0.44460,0.6566
1,1,rs6687776,1030565,T,ADD,777,1.3630,1.41600,0.1568
2,1,rs9651273,1031540,A,ADD,776,0.8177,-0.96420,0.3350
3,1,rs4970405,1048955,G,ADD,774,1.0040,0.01672,0.9867
4,1,rs12726255,1049950,G,ADD,777,1.1920,0.77210,0.4401


### SUR R
d= read.table("GRIV_DESIR_low_ld_pruned.pca.evec")

plot(d$V2, d$V3, pch=16)

library(qqman)
########## code visualistaion donnée R

 ############# awk '{print$9}' GRIV_DESIR_PR_nocovar.assoc.logistic > pvalue_assoc_grivPR_nocovar.txt
 
 t_3=read.table("~/stage/GRIV/pvalue_assoc_grivPR_nocovar.txt",header=TRUE)
 
 png(filename = "grivPR_assoc_nocovar_qq.png")
 
 qq(pvector = t_3$P)
 
 dev.off()
 
############### awk '{print$9}' low_ld.assoc.logistic > pvalue_association_grivPR.txt

t_3=read.table("~/stage/GRIV/pvalue_association_grivPR.txt",header=TRUE)

png(filename = "grivPR_assoc_wcovar_qq.png")

qq(pvector = t_3$P)

dev.off()